In [1]:
# Feature extraction
import musicbrainzngs
import ast
# Data processing
import numpy as np
import csv
import pandas as pd
# import matplotlib.pyplot as plt

# Models
import sklearn.preprocessing as Pre
from sklearn.cluster import KMeans
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import BaggingRegressor

# PCA and k-fold validation
# from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

# Data Processing

In [2]:
fulltrain_file = '../musicrec-data/train.csv'
ktrain_file = '../musicrec-data/ktrain.csv'
kvalid_file = '../musicrec-data/kval.csv'

# change this to reflect different solutions
soln_file  = 'user_genres_median.csv'

# for full data, non kfold
train_file = fulltrain_file
test_file  = '../musicrec-data/test.csv'

# for kfold validation
# train_file = ktrain_file
# test_file = kvalid_file
# soln_file = 'disregard.csv'

In [5]:
# get length of fulltrain
with open(fulltrain_file, 'r') as fulltrain_fh:
    fulltrain_csv = csv.reader(fulltrain_fh, delimiter=',', quotechar='"')
    next(fulltrain_csv, None)
    
    data = list(fulltrain_csv)
    rowsoftrain = len(data)

In [10]:
# Separate fulltrain_file into training set and validation set (for kfold)
c = 0
with open(fulltrain_file, 'r') as fulltrain_fh:
    fulltrain_csv = csv.reader(fulltrain_fh, delimiter=',', quotechar='"')
    next(fulltrain_csv, None)

    with open(ktrain_file, 'w') as ktrain_fh:
        ktrain_csv = csv.writer(ktrain_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)

        with open(kvalid_file, 'w') as kvalid_fh:
            kvalid_csv = csv.writer(kvalid_fh,
                                   delimiter=',',
                                   quotechar='"',
                                   quoting=csv.QUOTE_MINIMAL)
            
            for row in fulltrain_csv:
                if (c < (rowsoftrain * 4 / 5)):
                    ktrain_csv.writerow(row)
                else:
                    kvalid_csv.writerow(row)
                c += 1

In [3]:
# Load the training data.
train_data = {}
Y_actual = {} # for testing at end
id = 1
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        if (row):
            user   = row[0]
            artist = row[1]
            plays  = int(row[2])
            Y_actual[id] = plays
            id += 1
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = plays

In [5]:
# Load the test data. Not necessary rn
# test_data = {}
# with open(test_file, 'r') as test_fh:
#     test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
#     next(test_csv, None)
#     for row in test_csv:
#         user   = row[1]
#         artist = row[2]
#         if not user in test_data:
#             test_data[user] = {}
#         test_data[user][artist] = 0

# Global Median

In [45]:
# Predict via the median number of plays.
# Compute the global median.
plays_array = []
for user, user_data in train_data.items():
    for artist, plays in user_data.items():
        plays_array.append(plays)
global_median = np.median(np.array(plays_array))
print("global median:", global_median)

global median: 118.0


In [46]:
# Write out test solutions.
Y_hat = {} # add this Y_hat line_id stuff to every solution
line_id = 1
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            soln_csv.writerow([id, global_median])
            Y_hat[line_id] = global_median
            line_id += 1

# User Medians

In [4]:
# Predict via the user-specific median.
# If the user has no data, use the global median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.items():
    user_plays = []
    for artist, plays in user_data.items():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [9]:
c = 0
medians_list = []
for k, v in user_medians.items():
    medians_list.append(v)
    if (v > 10000):
        print(k)
        assert(0 == 1)
medians_array = np.array(medians_list)


710183e9a50e4bd8ad61b24ac4b287f03d51b322


AssertionError: 

In [11]:
train_data['710183e9a50e4bd8ad61b24ac4b287f03d51b322']

{'00a9f935-ba93-4fc8-a33a-993abe9c936b': 25479,
 '20953548-6d06-4428-99a9-c284bfaa5514': 11683,
 '2ecbc483-dee4-442f-8ce7-f3ab31c73f87': 16080,
 '541f16f5-ad7a-428e-af89-9fa1b16d3c9c': 4569,
 '5dedf5cf-a598-4408-9556-3bf3f149f3ba': 5708,
 '7d093650-89be-4108-842b-ba7f5367504b': 11359,
 '83d91898-7763-47d7-b03b-b92132375c47': 5180,
 '85af0709-95db-4fbc-801a-120e9f4766d0': 16155,
 '881c5eec-ab10-4523-98cc-0abf43900065': 2206,
 '8bf9b24b-e802-4ab6-b342-b348e20b58d4': 18163,
 '8bfac288-ccc5-448d-9573-c33ea2aa5c30': 13772,
 'bbd80354-597e-4d53-94e4-92b3a7cb8f2c': 2652,
 'd0400f72-c2df-4cb3-81bd-72fa19aeaef8': 2310,
 'ef58d4c9-0d40-42ba-bfab-9186c1483edd': 11113,
 'f57e14e4-b030-467c-b202-539453f504ec': 21627}

In [ ]:
import matplotlib.pyplot as plt
plt.hist(medians_array)
plt.show()
# conclusion- idk actually

In [75]:
# Write out test solutions.
Y_hat = {} # add this Y_hat line_id stuff to every solution
line_id = 1
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                Y_hat[line_id] = user_medians[user]
            else:
                print("User", id, "not in training data.")
                Y_hat[line_id] = global_median
            soln_csv.writerow([id, Y_hat[line_id]])
            line_id += 1

# Sex Medians

In [ ]:
user_df = pd.read_csv('musicrec-data/profiles.csv')
sex_dict = {row['user']:row['sex'] for (ind, row) in user_df.iterrows()}
user_df.head()

In [ ]:
# Predict via the sex-specific median
# If sex not given, use global median
male_plays = []
female_plays = []
for user, user_data in train_data.items():
    try:
        sex = sex_dict[user]
    except KeyError:
        sex = 'u'
    for artist, plays in user_data.items():
        if sex == 'm':
            male_plays.append(plays)
        elif sex == 'f':
            female_plays.append(plays)
male_median = np.median(np.array(male_plays))
female_median = np.median(np.array(female_plays))
nan_median = np.median(np.array(nan_plays))

In [ ]:
# Write out test solutions.
Y_hat = {} # add this Y_hat line_id stuff to every solution
line_id = 1
alpha = 0.5
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            try:
                sex = sex_dict[user]
                if sex == 'm':
                    Y_hat[line_id] = (alpha * user_medians[user]) + ((1 - alpha) * male_median)
                elif sex == 'f':
                    Y_hat[line_id] = (alpha * user_medians[user]) + ((1 - alpha) * female_median)
                else:
                    Y_hat[line_id] = (alpha * user_medians[user]) + ((1 - alpha) * global_median)
            except KeyError:
                Y_hat[line_id] = global_median
            soln_csv.writerow([id, Y_hat[line_id]])
            line_id += 1

# User Genre Pairings

In [9]:
# Predict via the user-genre mean.
# If the user-genre pairing hasn't happened before, use the global median.
# Read in artist_genre_dict and user_genre_dict from csv's.
# Alternatively, one could create them below with Jenn's code
agdfile = 'genre_dicts/artist_genre2_dict.csv'
artist_genre_dict = {}
with open(agdfile, 'r') as agd_fh:
    agd_csv = csv.reader(agd_fh, delimiter=',', quotechar='"')
    next(agd_csv, None)
    for row in agd_csv:
        artist_id = row[0]
        artist_genre1 = row[1]
        artist_genre2 = row[2]
        artist_genre_dict[artist_id] = (artist_genre1, artist_genre2)
        # not sure if this correctly turns blank cells into Nones

In [12]:
ugdfile = '../musicrec-data/user_genre2_dict.csv'
user_genre_dict = {}
with open(ugdfile, 'r') as ugd_fh:
    ugd_csv = csv.reader(ugd_fh, delimiter=',', quotechar='"')
    next(ugd_csv, None)
    c = 0
    for row in ugd_csv:
        user_id = row[0]
        user_dict = ast.literal_eval(row[1])
        user_genre_dict[user_id] = user_dict

In [65]:
# Write out test solutions.
# Need to run previous block of code to get user_medians and global_median
Y_hat = {} # add this Y_hat line_id stuff to every solution
line_id = 1
alpha = 0.4
soln_file = 'topgenres.csv'
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            genres = artist_genre_dict[artist]
            
            if user in user_medians:
                no_genre_median = user_medians[user]
            else:
                no_genre_median = global_median
                
            user_genre_avg_plays = 0
            ug_denom = 0
            # Sanity check if statements
            for genre in genres:
                if (genre in user_genre_dict[user]):
                    user_genre_avg_plays += user_genre_dict[user][genre][0]
                    ug_denom += user_genre_dict[user][genre][1]
#             if (genre1 and genre1 in user_genre_dict[user]):
#                 user_genre_avg_plays += user_genre_dict[user][genre1][0] / user_genre_dict[user][genre1][1]
#             if (genre2 and genre2 in user_genre_dict[user]):
#                 user_genre_avg_plays += user_genre_dict[user][genre2][0] / user_genre_dict[user][genre2][1]
#             if (genre1 and genre2 and genre1 in user_genre_dict[user] and genre2 in user_genre_dict[user]):
#                 user_genre_avg_plays /= 2

            if (user_genre_avg_plays):
                Y_hat[line_id] = alpha * (user_genre_avg_plays / ug_denom) + (1 - alpha) * no_genre_median
            else:
                Y_hat[line_id] = no_genre_median
            
            soln_csv.writerow([id, Y_hat[line_id]])
            line_id += 1

In [66]:
line_id

4154805

# Feature Extraction

In [5]:
# must set user agent before any requests are made
musicbrainzngs.set_useragent(1,1)

In [53]:
musicbrainzngs.get_artist_by_id('a74b1b7f-71a5-4011-9441-d0b5e4122711',includes=['tags'])

{'artist': {'area': {'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
   'iso-3166-1-code-list': ['GB'],
   'name': 'United Kingdom',
   'sort-name': 'United Kingdom'},
  'begin-area': {'id': 'd840d4b3-8987-4626-928b-398de760cc24',
   'name': 'Abingdon-on-Thames',
   'sort-name': 'Abingdon-on-Thames'},
  'country': 'GB',
  'id': 'a74b1b7f-71a5-4011-9441-d0b5e4122711',
  'isni-list': ['0000000115475162'],
  'life-span': {'begin': '1991'},
  'name': 'Radiohead',
  'sort-name': 'Radiohead',
  'tag-list': [{'count': '7', 'name': 'alternative rock'},
   {'count': '1', 'name': 'art pop'},
   {'count': '5', 'name': 'art rock'},
   {'count': '0', 'name': 'bootleg'},
   {'count': '0', 'name': 'britannique'},
   {'count': '3', 'name': 'british'},
   {'count': '2', 'name': 'britpop'},
   {'count': '-1', 'name': 'c’était mieux avant'},
   {'count': '4', 'name': 'electronic'},
   {'count': '0', 'name': 'england'},
   {'count': '1', 'name': 'english'},
   {'count': '2', 'name': 'experimental rock'},
  

In [51]:
# function for getting the mode tag for an artist (alphabetical order)
def modes_tag(artist_id):
    result = musicbrainzngs.get_artist_by_id(artist_id,includes=['tags'])
    tags = result['artist']['tag-list']
    genres = {}
    for genre in tags:
            keywords = (genre['name']).lower()
            keywords = keywords.replace('-',' ').replace('/',' ').split(' ')
            for keyword in keywords:
                if keyword in most_popular:
                    genres[keyword] = 1
    return genres

modes_tag('a74b1b7f-71a5-4011-9441-d0b5e4122711')

{'alternative': 1,
 'art': 1,
 'britannique': 1,
 'british': 1,
 'electronic': 1,
 'england': 1,
 'english': 1,
 'experimental': 1,
 'grunge': 1,
 'indie': 1,
 'pop': 1,
 'rock': 1,
 'uk': 1}

In [9]:
# read in artist information
artists = pd.read_csv('../musicrec-data/artists.csv', sep=',')
artist_ids = artists['artist'].values

In [54]:
# build artist_genre_dict
artist_genre_dict = {}
c = 0
for artist_id in artist_ids:
    try: 
        genres = modes_tag(artist_id)
        artist_genre_dict[artist_id] = genres
    except (KeyError, musicbrainzngs.ResponseError):
        artist_genre_dict[artist_id] = {} #(None, None)
    c += 1
    if (c % 100 == 0):
        print(genres)

# takes forever

{'baroque': 1, 'progressive': 1, 'art': 1, 'rock': 1, 'english': 1, 'pop': 1, 'british': 1, 'classic': 1, 'symphonic': 1, 'uk': 1}
{'rock': 1, 'english': 1, 'indie': 1, 'british': 1, 'uk': 1}
{'singer': 1, 'piano': 1, 'pop': 1, 'british': 1, 'soft': 1, 'folk': 1, 'acoustic': 1, 'rock': 1, 'english': 1, 'uk': 1, 'songwriter': 1, 'chart': 1}
{'post': 1, 'punk': 1, 'new': 1, 'british': 1, 'pop': 1, 'england': 1, 'alternative': 1, 'baroque': 1, 'classic': 1, 'rock': 1, 'wave': 1, 'roll': 1, 'uk': 1}
{'blues': 1, 'rock': 1, 'death': 1, 'jazz': 1}
{'rock': 1, 'female': 1, 'metal': 1, 'new': 1, 'post': 1, 'alternative': 1, 'grunge': 1, 'hard': 1, 'christian': 1}
{'american': 1, 'thrash': 1, 'metal': 1, 'metalcore': 1, 'hardcore': 1}
{}
{'female': 1, 'electropop': 1, 'synthpop': 1}
{'irish': 1, 'norwegian': 1, 'folk': 1}
{'pop': 1, 'chart': 1}
{'singer': 1, 'female': 1, 'swedish': 1, 'rock': 1, 'pop': 1, 'songwriter': 1}
{'80s': 1, 'progressive': 1, 'art': 1, 'opera': 1, 'pop': 1, 'british': 1

In [56]:
for k,v in artist_genre_dict.items():
    print(k)
    print(v)
    break
print(len(artist_genre_dict))

3e55d51d-687f-4a9d-af96-2fabccf802e5
{'indie': 1, 'rock': 1, 'swedish': 1, 'alternative': 1, 'european': 1, 'pop': 1}
2000


In [58]:
# SAVE AS CSV
with open('artist_topgenre_dict_old.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in artist_genre_dict.items():
        writer.writerow([key, value])

In [62]:
user_genre_dict = {}
# train_data is a dict user: (dict artist: plays)
for user, artist_info in train_data.items():
    if (artist_info == (None, None)):
        pass
    genre_dict = {}
    for artist, plays in artist_info.items():
        gen_labels = artist_genre_dict[artist]
        for genre in gen_labels:
            if genre not in genre_dict:
                genre_dict[genre] = (plays, 1)
            else:
                (num_genre_plays, num_genre_artists) = genre_dict[genre]
                genre_dict[genre] = (num_genre_plays + plays, num_genre_artists + 1)
#         # only proceed if genre is not None
#         if genre1:
#             # if the user hasn't already listened to the genre
#             if genre1 not in genre_dict:
#                 genre_dict[genre1] = (plays, 1)
#             # otherwise, add number of plays to current total
#             else:
#                 (num_genre_plays, num_genre_artists) = genre_dict[genre1]
#                 genre_dict[genre1] = (num_genre_plays + plays, num_genre_artists + 1)
#         if genre2:
#             if genre2 not in genre_dict:
#                 genre_dict[genre2] = (plays, 1)
#             else:
#                 (num_genre_plays, num_genre_artists) = genre_dict[genre2]
#                 genre_dict[genre2] = (num_genre_plays + plays, num_genre_artists + 1)
    user_genre_dict[user] = genre_dict

In [63]:
for k,v in user_genre_dict.items():
    print(k)
    print(v)
    break
print(len(user_genre_dict))

b033ce37c1cd4c442cd12a595ec3adcd1061111c
{'american': (500, 3), 'piano': (112, 1), 'progressive': (338, 1), 'united': (887, 1), 'emo': (195, 1), 'r&b': (540, 1), 'alternative': (511, 3), 'metal': (314, 2), '10s': (272, 1), '90s': (272, 1), 'soul': (116, 1), 'uk': (887, 1), 'house': (494, 2), 'christian': (195, 1), 'songwriter': (276, 1), 'score': (267, 1), 'downtempo': (887, 1), 'french': (267, 1), 'film': (267, 1), 'seen': (397, 2), 'electronica': (1038, 6), '00s': (272, 1), 'américain': (116, 1), 'dance': (2501, 9), 'symphonic': (42, 1), 'trance': (687, 4), 'hall': (540, 1), '80s': (272, 1), 'pop': (2167, 7), 'electro': (267, 1), 'live': (397, 2), 'synthpop': (1154, 2), 'european': (433, 4), 'vocal': (401, 2), 'reggae': (116, 1), 'folk': (816, 2), 'classic': (1163, 2), 'techno': (44, 1), 'hip': (847, 3), 'nu': (267, 1), 'chart': (892, 4), 'dutch': (666, 4), 'singer': (1163, 2), 'gothic': (42, 1), 'electropop': (887, 1), 'canadian': (540, 1), 'electronic': (660, 5), 'german': (63, 1),

In [18]:
# SAVE AS CSV
with open('user_topgenre_dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in user_genre_dict.items():
        writer.writerow([key, value])

## Artist information, searching, and browsing

In [44]:
musicbrainzngs.search_artists("radiohead")
#musicbrainzngs.search_recordings("bennie and the jets")

radiohead_id = "a74b1b7f-71a5-4011-9441-d0b5e4122711"
radiohead_result = musicbrainzngs.get_artist_by_id(radiohead_id,includes=["releases","ratings"])
radiohead_artist = radiohead_result["artist"]

print("name:\t\t%s" % radiohead_artist["name"])
print("rating:\t\t%s" % radiohead_artist["rating"])
print("release 0:\t%s" % radiohead_artist["release-list"][0])

# dictionary of recordings for radiohead
# musicbrainzngs.browse_recordings(radiohead_id)

# "artist" can include the following information:
# recordings, releases, release-groups, works, various-artists,
# discids, media, isrcs, aliases, annotation, area-rels, artist-rels,
# label-rels, place-rels, event-rels, recording-rels, release-rels,
# release-group-rels, series-rels, url-rels, work-rels, instrument-rels,
# tags, user-tags, ratings, user-ratings

name:		Radiohead
rating:		{'votes-count': '59', 'rating': '4.45'}
release 0:	{'release-event-list': [{'date': '1993', 'area': {'id': '89a675c2-3e37-3518-b83c-418bad59a85a', 'name': 'Europe', 'iso-3166-1-code-list': ['XE'], 'sort-name': 'Europe'}}], 'text-representation': {'language': 'eng', 'script': 'Latn'}, 'title': 'Pablo Honey', 'id': '06487940-c7d4-4af3-976d-0be796d686ce', 'packaging': 'Jewel Case', 'date': '1993', 'country': 'XE', 'status': 'Official', 'release-event-count': 1, 'quality': 'normal', 'barcode': '077778140924'}


## Cover art information

In [42]:
# gets the list of cover art associated with a release
musicbrainzngs.get_image_list("081e7033-5282-4bd7-9963-e64edf8c693a")

# downloads the front cover art of a release
# musicbrainzngs.get_image_front("081e7033-5282-4bd7-9963-e64edf8c693a")

{u'images': [{u'approved': True,
   u'back': False,
   u'comment': u'',
   u'edit': 35299266,
   u'front': True,
   u'id': 11628918400,
   u'image': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400.jpg',
   u'thumbnails': {u'large': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400-500.jpg',
    u'small': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400-250.jpg'},
   u'types': [u'Front']}],
 u'release': u'https://musicbrainz.org/release/081e7033-5282-4bd7-9963-e64edf8c693a'}

In [15]:
aid = 'f57e14e4-b030-467c-b202-539453f504ec'
musicbrainzngs.get_artist_by_id(aid,includes=["tags"])

{'artist': {'area': {'id': '6a264f94-6ff1-30b1-9a81-41f7bfabd616',
   'iso-3166-1-code-list': ['FI'],
   'name': 'Finland',
   'sort-name': 'Finland'},
  'begin-area': {'id': '638a021d-f628-4266-8a91-7956f91f1e08',
   'name': 'Espoo',
   'sort-name': 'Espoo'},
  'country': 'FI',
  'id': 'f57e14e4-b030-467c-b202-539453f504ec',
  'isni-list': ['0000000122931058'],
  'life-span': {'begin': '1997'},
  'name': 'Children of Bodom',
  'sort-name': 'Children of Bodom',
  'tag-list': [{'count': '1', 'name': 'death metal'},
   {'count': '2', 'name': 'finnish'},
   {'count': '4', 'name': 'melodic death metal'},
   {'count': '2', 'name': 'metal'},
   {'count': '0', 'name': 'rock and indie'}],
  'type': 'Group'}}

# Making Predictions

In [ ]:
# Write out test solutions.
# Need to run previous block of code to get user_medians and global_median
Y_hat = {} # add this Y_hat line_id stuff to every solution
line_id = 1
alpha = 0.4
soln_file = 'topgenres.csv'
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            genres = artist_genre_dict[artist]
            
            if user in user_medians:
                no_genre_median = user_medians[user]
            else:
                no_genre_median = global_median
                
            user_genre_avg_plays = 0
            ug_denom = 0
            # Sanity check if statements
            for genre in genres:
                if (genre in user_genre_dict[user]):
                    user_genre_avg_plays += user_genre_dict[user][genre][0]
                    ug_denom += user_genre_dict[user][genre][1]
#             if (genre1 and genre1 in user_genre_dict[user]):
#                 user_genre_avg_plays += user_genre_dict[user][genre1][0] / user_genre_dict[user][genre1][1]
#             if (genre2 and genre2 in user_genre_dict[user]):
#                 user_genre_avg_plays += user_genre_dict[user][genre2][0] / user_genre_dict[user][genre2][1]
#             if (genre1 and genre2 and genre1 in user_genre_dict[user] and genre2 in user_genre_dict[user]):
#                 user_genre_avg_plays /= 2

            if (user_genre_avg_plays):
                Y_hat[line_id] = alpha * (user_genre_avg_plays / ug_denom) + (1 - alpha) * no_genre_median
            else:
                Y_hat[line_id] = no_genre_median
            
            soln_csv.writerow([id, Y_hat[line_id]])
            line_id += 1

In [73]:
# For kfold: make predictions on partial training data (using user median criteria)
Y_hat = {}
line_id = 1
alpha = 0.8
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for row in train_csv:
        user = row[0]
        artist = row[1]
        genres = artist_genre_dict[artist]
        # modify this part if using different criteria
        if user in user_medians:
            no_genre_median = user_medians[user]
        else:
            no_genre_median = global_median
                
        user_genre_avg_plays = 0
        ug_denom = 0
        # Sanity check if statements
        for genre in genres:
            if (genre in user_genre_dict[user]):
                user_genre_avg_plays += user_genre_dict[user][genre][0]
                ug_denom += user_genre_dict[user][genre][1]
                    
        if (user_genre_avg_plays):
            Y_hat[line_id] = alpha * (user_genre_avg_plays / ug_denom) + (1 - alpha) * no_genre_median
        else:
            Y_hat[line_id] = no_genre_median
        line_id += 1

In [77]:
np.mean([abs(Y_hat[i] - Y_actual[i]) for i in range(1, len(Y_hat))])

254.86761466187446

# K Means

In [6]:
# must set user agent before any requests are made
# musicbrainzngs.set_useragent(1,1)
# top 100 words
artists = pd.read_csv('../musicrec-data/artists.csv', sep=',')
artist_ids = artists['artist'].values

In [14]:
most_popular = {}
c = 0
for artist_id in artist_ids[500:]:
    try: 
        result = musicbrainzngs.get_artist_by_id(artist_id,includes=['tags'])
        tags = result['artist']['tag-list']
        for genre in tags:
            keywords = (genre['name']).lower()
            keywords = keywords.replace('-',' ').replace('/',' ').split(' ')
            for keyword in keywords:
                if (keyword in most_popular.keys()):
                    most_popular[keyword] += 1
                else:
                    most_popular[keyword] = 1
    except (KeyError, musicbrainzngs.ResponseError):
        pass
    
    c += 1
    if (c % 20 == 0):
        print(c)
    if (c > 500):
        break
        
print(most_popular)
    
# parse strings
# get set of top 100
# create a dict just like the last one
# 16

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
{'': 4, 'electro': 13, 'glitch': 1, 'sillyname': 3, 'british': 71, 'thrash': 10, 'unknown': 1, 'r&b': 18, 'explain': 1, 'nu': 8, 'grooves': 1, 'house': 18, "80's": 1, 'dream': 4, 'score': 4, 'composer': 15, 'swiss': 2, 'space': 2, 'protopunk': 1, '*england→north': 1, 'poppunk': 1, 'lazy': 2, 'mess!': 1, 'electronica': 42, 'southern': 5, 'shock': 1, 'beyer': 1, '333': 1, 'definitive': 1, 'dance': 89, 'grindcore': 1, 'hall': 16, 'dedicated': 1, 'espagne': 1, 'nitzer': 1, 'base': 1, 'рождества': 1, 'catalana': 1, 'symphony': 1, 'bass': 10, '51': 1, 'black': 10, 'kind': 1, 'honky': 1, 'brutal': 1, 'oldest': 1, 'doo': 1, 'failure': 1, 'ffs': 1, 'orchestral': 1, 'avant': 4, '70s': 5, 'lies': 1, 'up': 3, 'ultimatecore': 1, 'meltdown': 1, 'chamber': 1, 'roll': 9, '34ad': 1, 'american': 90, 'casanovas': 1, 'nashville': 1, 'summer': 1, 'manson': 1, 'méxico': 3, 'brésilien': 1, 'country': 25, 'berlin':

In [15]:
print({k: v for k, v in most_popular.items() if v > 20})

{'american': 90, 'pop': 379, 'power': 26, 'punk': 94, 'post': 48, 'new': 41, 'german': 30, 'uk': 61, 'jazz': 35, 'heavy': 21, 'hard': 32, 'metal': 235, 'folk': 66, 'soul': 34, 'classic': 88, 'alternative': 99, 'english': 48, 'blues': 50, 'chart': 24, 'singer': 22, 'gothic': 22, 'country': 25, 'hip': 79, 'progressive': 37, 'hardcore': 28, 'electronic': 49, 'british': 71, 'rap': 24, 'electronica': 42, 'indie': 148, 'hop': 89, 'death': 36, 'usa': 44, 'rock': 705, 'and': 248, 'contemporary': 25, 'américain': 21, 'dance': 89, 'rnb': 23, 'experimental': 21}


In [50]:
with open('mostmostpopular.csv', 'w') as mp_fh:
    mp_csv = csv.writer(mp_fh, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for k,v in most_popular.items():
        if (v > 1):
            mp_csv.writerow([k,v])

In [19]:
with open('mostpopular.csv', 'r') as mp_fh:
    mp_csv = csv.reader(mp_fh, delimiter=',', quotechar='"')
    next(mp_csv, None)
    
    for row in mp_csv:
        if (row):
            word = row[0]
            number = int(row[1])
            if word in most_popular:
                most_popular[word] += number
            else:
                most_popular[word] = number

In [47]:
most_popular = {k: v for k, v in most_popular.items() if v > 9}

In [49]:
len(most_popular)

{'00s': 15,
 '10s': 10,
 '70s': 11,
 '80s': 27,
 '90s': 15,
 'acoustic': 35,
 'adult': 19,
 'age': 10,
 'alternative': 230,
 'ambient': 34,
 'american': 175,
 'américain': 31,
 'aor': 19,
 'art': 38,
 'artist': 16,
 'australian': 10,
 'band': 21,
 'baroque': 11,
 'bass': 13,
 'black': 20,
 'blues': 92,
 'britannique': 32,
 'british': 155,
 'california': 12,
 'canadian': 23,
 'celtic': 10,
 'chart': 63,
 'christian': 10,
 'christmas': 12,
 'classic': 163,
 'classical': 30,
 'coast': 18,
 'composer': 41,
 'contemporary': 56,
 'country': 50,
 'dance': 188,
 'dark': 10,
 'death': 70,
 'disco': 18,
 'doom': 12,
 'downtempo': 19,
 'dream': 12,
 'drum': 12,
 'dutch': 10,
 'east': 10,
 'easy': 10,
 'electro': 20,
 'electronic': 111,
 'electronica': 82,
 'electropop': 37,
 'emo': 19,
 'england': 26,
 'english': 105,
 'euro': 13,
 'european': 41,
 'experimental': 37,
 'female': 17,
 'film': 15,
 'folk': 128,
 'french': 21,
 'funk': 26,
 'garage': 18,
 'german': 52,
 'glam': 22,
 'gothic': 34,
 '

In [44]:
del most_popular['the']
del most_popular['this']
del most_popular['of']
del most_popular['by']
del most_popular['and']
del most_popular['&']

In [ ]:
# Jenn's old code
# function for getting the mode tag for an artist (alphabetical order)
def mode_tag(artist_id):
    result = musicbrainzngs.get_artist_by_id(artist_id,includes=['tags'])
    tags = result['artist']['tag-list']
    mode_tag = None
    mode = None
    for tag_dict in tags:
        if tag_dict['count'] > mode:
            mode = tag_dict['count']
            mode_tag = tag_dict['name']
    return mode_tag

# mode_tag('a74b1b7f-71a5-4011-9441-d0b5e4122711')
# read in artist information
artists = pd.read_csv('../musicrec-data/artists.csv', sep=',')
artist_ids = artists['artist'].values

# build artist_genre_dict
artist_genre_dict = {}
for artist_id in artist_ids:
    print artist_id
    # update artist_genre dict
    try: 
        genre = mode_tag(artist_id)
        artist_genre_dict[artist_id] = genre
    except (KeyError, musicbrainzngs.ResponseError):
        artist_genre_dict[artist_id] = None
        
user_genre_dict = {}
# train_data is a dict user: (dict artist: plays)
for user, artist_info in train_data.items():
    genre_dict = {}
    for artist, plays in artist_info.items():
        genre = artist_genre_dict[artist]
        # only proceed if genre is not None
        if genre:
            # if the user hasn't already listened to the genre
            if genre not in genre_dict:
                genre_dict[genre] = (plays, 1)
            # otherwise, add number of plays to current total
            else:
                (num_genre_plays, num_genre_artists) = genre_dict[genre]
                genre_dict[genre] = (num_genre_plays + plays, num_genre_artists + 1)
    user_genre_dict[user] = genre_dict
    
for k,v in artist_genre_dict.items():
    print(k)
    print(v)
    break
print(len(artist_genre_dict))
for k,v in user_genre_dict.items():
    print(k)
    print(v)
    break
print(len(user_genre_dict))

# SAVE AS CSV
with open('artist_genre_dict.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in artist_genre_dict.items():
        if value:
            writer.writerow([key, value.encode('ascii', 'ignore').decode('ascii')])
        else:
            writer.writerow([key, value])
with open('user_genre_dict.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in user_genre_dict.items():
        writer.writerow([key, value])